# Solver

In [1]:
from numpy import reshape, genfromtxt, loadtxt, unique, delete, where, concatenate, logical_or,asarray, array, empty, copy, append
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [8]:
def findCars(board):
    cars = delete(unique(board),[0])
    movement = empty(cars.size, dtype = 'str')
    for i, car in enumerate(cars):
        index = where(board == car)
        if unique(index[0]).size == 1:
            movement[i] = 'h'
        else:
            movement[i] = 'v'
    return (cars,movement)

def canItMoveHorizontally(board,mov):
    position = list(where(board == mov[0]))
    if mov[1] == 'left':
        position = [position[0],position[1]-1]
    else:
        position = [position[0],position[1]+1]
        
    if not((position[1]>-1).all() and (position[1]<6).all()) or not(logical_or(board[position] == '.', board[position] == mov[0]).all()):
        return False
    return True

def canItMoveVertically(board,mov):
    position = list(where(board == mov[0]))
    if mov[1] == 'up':
        position = [position[0]-1,position[1]]
    else:
        position = [position[0]+1,position[1]]
        
    if not((position[0]>-1).all() and (position[0]<6).all()) or not(logical_or(board[position] == '.', board[position] == mov[0]).all()):
        return False
    return True

def possibleMovements(board):
    cars, directions = findCars(board)
    possible_movements = []
    for i,car in enumerate(cars):
        if directions[i] == 'h':
            possible_movements.extend([(car,direction) for direction in ['left','right'] if canItMoveHorizontally(board,(car,direction))])                
        else:
            possible_movements.extend([(car,direction) for direction in ['up','down'] if canItMoveVertically(board,(car,direction))]) 
    return possible_movements

def move(board,mov):
    board_aux = copy(board)
    position = list(where(board_aux == mov[0]))
    board_aux[position] = '.'
    if mov[1] == 'left':
        position = [position[0],position[1]-1]
    elif mov[1] == 'right':
        position = [position[0],position[1]+1]
    elif mov[1] == 'up':
        position = [position[0]-1,position[1]]
    else:
        position = [position[0]+1,position[1]] 
    board_aux[position] = mov[0]
    return board_aux

def addBoard(boards_database,board, num = 0):
    if not(any((board == saved_board).all() for saved_board in boards_database[-num:])):
        boards_database.append(board)
        return True
    return False
        
def isSolution(board):
    position = where(board == 'r')
    if (board[position[0],max(position[1])+1:]== '.').all():
        return True
    return False


def allMoves(in_board,max_num_moves,crosscheck_steps):
    if isSolution(in_board):
        print 'No need for any movements, just drive out!!'
        return
    boards_database = []
    addBoard(boards_database,in_board)
    boards = [in_board]
    sequences = empty((1,0),dtype = 'object')
    count = [1]
    for num_move in range(max_num_moves):
        #print 'Looking for solution in %d steps' %(num_move+1)
        boards_aux = []
        sequences_aux  = empty((0,2*(num_move+1))) 
        num_count = 0
        for i, board in enumerate(boards):
            for j,mov in enumerate([x for x in possibleMovements(board) if addBoard(boards_database,move(board,x),sum(count[-crosscheck_steps:]))]):
                num_count += 1
                if isSolution(move(board,mov)):
                    print 'Number of moves: %d' %(num_move+1) 
                    seq = sequences[i,:]
                    return (append(seq,mov),move(board,mov),True,len(boards_database),move(board,mov))
                boards_aux.append(move(board,mov))
                seq_row = append(sequences[i,:],mov)
                seq_row.shape = (1,len(seq_row))
                sequences_aux = append(sequences_aux, seq_row, axis = 0)
        sequences = sequences_aux
        boards = boards_aux
        count.append(num_count)
    return (sequences,boards, False,len(boards_database))


In [9]:
in_board = genfromtxt('advanced.txt', delimiter=',',dtype='str')

max_num_moves = 100

crosscheck_steps = 8 #Set to 0 if you want to crosscheck repetition against all the boards database


in_time = time.clock()
sequence, final_board, is_solved, num_boards, final_board = allMoves(in_board,max_num_moves, crosscheck_steps)
fin_time = time.clock()
total_time = fin_time - in_time;
    
if is_solved:
    print 'Execution time: %.2f seconds' %total_time
    print 'The number of boards in the board_database is: %d' %num_boards
    print 'Initial board:   Final board:'
    
    for i in range(6):
        print " {0},{1},{2},{3},{4},{5}".format(*in_board[i])+ "      " + " {0},{1},{2},{3},{4},{5}".format(*final_board[i])
    print 'The winning sequence is:'
    print sequence
    print 'Drive Out! Tot ziens.'
else:
    print 'Solution not found in %d moves.'%max_num_moves

Number of moves: 45
Execution time: 61.78 seconds
The number of boards in the board_database is: 2020
Initial board:   Final board:
 .,.,A,A,A,B       .,.,C,A,A,A
 .,.,C,D,D,B       D,D,C,.,.,.
 .,.,C,r,r,B       r,r,.,.,.,.
 .,.,F,G,H,H       H,H,F,G,.,B
 .,.,F,G,I,I       I,I,F,G,.,B
 .,.,J,J,J,.       .,.,J,J,J,B
The winning sequence is:
['J' 'left' 'J' 'left' 'G' 'down' 'H' 'left' 'B' 'down' 'A' 'right' 'C'
 'up' 'r' 'left' 'r' 'left' 'r' 'left' 'C' 'down' 'A' 'left' 'B' 'up' 'H'
 'right' 'G' 'up' 'G' 'up' 'I' 'left' 'J' 'right' 'J' 'right' 'J' 'right'
 'F' 'down' 'C' 'down' 'D' 'left' 'D' 'left' 'D' 'left' 'C' 'up' 'G' 'up'
 'H' 'left' 'B' 'down' 'A' 'right' 'B' 'down' 'C' 'up' 'H' 'left' 'H'
 'left' 'G' 'down' 'H' 'left' 'F' 'up' 'F' 'up' 'I' 'left' 'I' 'left' 'G'
 'down' 'I' 'left' 'F' 'down' 'J' 'left' 'B' 'down']
Drive Out! Tot ziens.


# Solution Visualization

In [4]:
from pylab import imshow, show, subplot,figure
from numpy import ones, array,sqrt

In [5]:
def toColor(board,cars,colors):
    board_aux = ones((board.shape))
    for i,car in enumerate(cars):
        board_aux[where(board==car)] = colors[i]
    board_aux[where(board=='.')] = max(colors)+1
    return board_aux

def generateBoards(in_board,sequence):
    boards = []
    boardsColor = []
    cars,_ = findCars(in_board)
    c = sorted(cars)
    colors = range(len(sorted(cars)))
    colors[where(cars=='r')[0]] = min(colors)-2
    boards.append(in_board)
    boardsColor.append(toColor(in_board,cars,colors))
    
    seq = [(sequence[2*i],sequence[2*i+1]) for i in range(len(sequence)/2)]
    
    for i,mov in enumerate(seq):
        boards.append(move(boards[i],mov))
        boardsColor.append(toColor(boards[i+1],cars,colors))
    return boardsColor

def plotBoards(boards,seq):
    s = len(boards)/10+1
    fig = figure()
    for i,board in enumerate(boards[1:]):
        figr = fig.add_subplot(s,10,i+1)
        figr.axes.get_xaxis().set_visible(False)
        figr.axes.get_yaxis().set_visible(False)
        #if i>0: 
        figr.set_title('Step #%d'%(i+1),fontsize = 20)
        #else:
            #figr.set_title('Initial Board',fontsize = 20)
        imshow(board, cmap = 'cubehelix',interpolation='none')
    fig.subplots_adjust(hspace=.3)
    fig.subplots_adjust(wspace=0.001)
    fig.suptitle('The BLACK car is the one to drive out \nExecution Time: %.2fsec    Cross Check Steps: %d'%(total_time,crosscheck_steps), fontsize = 20)
    show()

In [7]:
%matplotlib qt
color_boards = generateBoards(in_board,sequence)
plotBoards(color_boards,sequence)